# Levenshtein Example (strings data set)
In this notebook we use levenshtein (edit distance) to compare strings from annchors string data set.

In [1]:
# requires python-levenshtein (pip install python-Levenshtein)
# requires nmslib (pip install nmslib)
import numpy as np
import time
import Levenshtein as lev
import os

from annchor import Annchor, BruteForce, compare_neighbor_graphs
from annchor.datasets import load_strings
from annchor import compare_neighbor_graphs

In [2]:
k=15

strings_data = load_strings()
X = strings_data['X']
y = strings_data['y']
neighbor_graph = strings_data['neighbor_graph']

nx = X.shape[0]

for x in X[::100]:
    print(x[:50]+'...')

cuiojvfnseoksugfcbwzrcoxtjxrvojrguqttjpeauenefmkmv...
uiofnsosungdgrxiiprvojrgujfdttjioqunknefamhlkyihvx...
cxumzfltweskptzwnlgojkdxidrebonxcmxvbgxayoachwfcsy...
cmjpuuozflodwqvkascdyeosakdupdoeovnbgxpajotahpwaqc...
vzdiefjmblnumdjeetvbvhwgyasygrzhuckvpclnmtviobpzvy...
nziejmbmknuxdhjbgeyvwgasygrhcpdxcgnmtviubjvyzjemll...
yhdpczcjxirmebhfdueskkjjtbclvncxjrstxhqvtoyamaiyyb...
yfhwczcxakdtenvbfctugnkkkjbcvxcxjwfrgcstahaxyiooeb...
yoftbrcmmpngdfzrbyltahrfbtyowpdjrnqlnxncutdovbgabo...
tyoqbywjhdwzoufzrqyltahrefbdzyunpdypdynrmchutdvsbl...
dopgwqjiehqqhmprvhqmnlbpuwszjkjjbshqofaqeoejtcegjt...
rahobdixljmjfysmegdwyzyezulajkzloaxqnipgxhhbyoztzn...
dfgxsltkbpxvgqptghjnkaoofbwqqdnqlbbzjsqubtfwovkbsk...
pjwamicvegedmfetridbijgafupsgieffcwnmgmptjwnmwegvn...
ovitcihpokhyldkuvgahnqnmixsakzbmsipqympnxtucivgqyi...
xvepnposhktvmutozuhkbqarqsbxjrhxuumofmtyaaeesbeuhf...


## Using Annchor

Let's see how we use annchor to find the k-NN graph for this data set.

Levenshtein distance is included in Annchor, we can simply specify the metric by the string 'levenshtein'.
We will use 23 anchor points, a sample size of 5000, and aim to use only 12% of the work required to brute force the solution.



(Remember that the first time we run annchor will be longer than usual due to the numba.jit compile time overhead, so run this cell twice to get a good idea of timings)

In [4]:
start_time = time.time()

ann = Annchor(X,
              'levenshtein',
              n_anchors=23,
              n_neighbors=k,
              random_seed=5,
              n_samples=5000,
              p_work=0.12,
              niters=4)

ann.fit()
print('ANNchor Time: %5.3f seconds' % (time.time()-start_time))


# Test accuracy
error = compare_neighbor_graphs(neighbor_graph,
                                ann.neighbor_graph,
                                k)
print('ANNchor Accuracy: %d incorrect NN pairs (%5.3f%%)' % (error,100*error/(k*nx)))

ANNchor Time: 28.269 seconds
ANNchor Accuracy: 0 incorrect NN pairs (0.000%)


## Comparison with other techniques
Now compare this to Brute Force, or the nmslib library (Annchor comes with a built in brute force option).

### Brute Force
The next cell uses annchors brute force implimentation (which is parallelised by default)

In [5]:
start_time = time.time()

bruteforce = BruteForce(X,'levenshtein')
bruteforce.fit()

print('Brute Force Time: %5.3f seconds' % (time.time()-start_time))

error = compare_neighbor_graphs(neighbor_graph,
                                bruteforce.neighbor_graph,
                                k)

print('Brute Force Accuracy: %d incorrect NN pairs (%5.3f%%)' % (error,100*error/(k*nx)))

Brute Force Time: 211.568 seconds
Brute Force Accuracy: 0 incorrect NN pairs (0.000%)


### nmslib

In [6]:
import nmslib

start_time = time.time()

# specify some parameters
CPU_COUNT = os.cpu_count()

index_time_params = {'M': 20,
                     'indexThreadQty': CPU_COUNT,
                     'efConstruction': 100,
                     'post' : 2}

# create the index
index = nmslib.init(method='hnsw',
                    space='leven',
                    dtype=nmslib.DistType.INT,
                    data_type=nmslib.DataType.OBJECT_AS_STRING)

index.addDataPointBatch(data=list(X))
index.createIndex(index_time_params,print_progress=True)

# query the index
res = index.knnQueryBatch(list(X), k=k, num_threads=CPU_COUNT)
hnsw_neighbor_graph = [np.array([x[0]for x in res]),np.array([x[1]for x in res])]
print('HNSW Time: %5.3f seconds' % (time.time()-start_time))


error = compare_neighbor_graphs(neighbor_graph,
                                hnsw_neighbor_graph,
                                k)

print('HNSW Accuracy: %d incorrect NN pairs (%5.3f%%)' % (error,100*error/(k*nx)))

HNSW Time: 312.774 seconds
HNSW Accuracy: 12 incorrect NN pairs (0.050%)
